In [67]:
import pandas as pd
import numpy as np
from datetime import timedelta
import plotly.graph_objects as go

In [68]:
# Simular dados de uso
np.random.seed(42)
users = [f"user_{i}" for i in range(1, 21)]

start_date = pd.to_datetime("2025-02-01")

data = []
for user in users:
    active_days = np.random.choice(range(90), size=np.random.randint(3, 6), replace=False)
    for day in active_days:
        data.append({"user_id": user, "date": start_date + timedelta(days=int(day))})

df = pd.DataFrame(data)
df["date"] = pd.to_datetime(df["date"])
df.head()

,user_id,date
0,user_1,2025-03-13
1,user_1,2025-02-23
2,user_1,2025-03-28
3,user_1,2025-04-12
4,user_1,2025-02-01


In [ ]:
# Calcular MAU mensal (mês-calendário)
# Agrupar por mês e contar usuários únicos
df["year_month"] = df["date"].dt.to_period("M")
mau_mensal = df.groupby("year_month")["user_id"].nunique().reset_index()
mau_mensal["year_month"] = mau_mensal["year_month"].dt.strftime("%b %Y")

In [ ]:
#  Plotar MAU mensal (mês-calendário)
fig_mau_calendar = go.Figure()

fig_mau_calendar.add_trace(go.Scatter(
    x=mau_mensal["year_month"],
    y=mau_mensal["user_id"],
    mode="lines+markers+text",
    text=mau_mensal["user_id"],
    textposition="top center",
    name="MAU mensal",
    line=dict(color="darkblue", width=2),
    marker=dict(symbol="circle", size=8)
))

fig_mau_calendar.update_layout(
    title="MAU por mês-calendário",
    xaxis_title="Mês",
    yaxis_title="Usuários Únicos",
    template="plotly_white"
)

fig_mau_calendar.show()

In [ ]:
# Calcular rolling MAU (últimos 30 dias)
rolling_mau = []
for dia in pd.date_range(start=df["date"].min(), end=df["date"].max()):
    janela_inicio = dia - timedelta(days=29)
    usuarios = df[(df["date"] >= janela_inicio) & (df["date"] <= dia)]["user_id"].nunique()
    rolling_mau.append({"date": dia, "mau": usuarios})

rolling_df = pd.DataFrame(rolling_mau)

In [ ]:
#  Plotar rolling MAU
fig_rolling_mau = go.Figure()

fig_rolling_mau.add_trace(go.Scatter(
    x=rolling_df["date"],
    y=rolling_df["mau"],
    mode="lines",
    # mode="lines+markers+text",
    name="MAU rolling",
    text=rolling_df["mau"],
    textposition="top center",
    line=dict(color="blue", width=2)
    # marker=dict(symbol="circle", size=6)
))

fig_rolling_mau.update_layout(
    title="MAU com janela rolante (últimos 30 dias)",
    xaxis_title="Data",
    yaxis_title="Usuários Únicos",
    template="plotly_white"
)

fig_rolling_mau.show()